In [66]:
import glob2
import json

In [67]:
def findEndIndex(string):
    count=0
    for i in range(len(string)):
        if ((string[i]=='{') or (string[i]=='(')):
            count=count+1
        if ((string[i]=='}') or (string[i]==')')):
            count=count-1
        if (count==0 and string[i]==";"):
            return i
    return None

In [68]:
def findNextParenthesis(string):
    count=0
    for i in range(len(string)):
        if (string[i]=='('):
            return i
    return None  

In [69]:
def findEndIndexXML(string):
    for i in range(len(string)):
        if (string[i]==">"):
            return i
    return None

In [70]:
def findEndIndexLine(string):
    for i in range(len(string)):
        if (string[i]=="\""):
            return i
    return None

In [71]:
# Récupération dans les opérateurs des préconditions

ListFile=glob2.glob(r".\rapidminer-studio-modular-master\rapidminer-studio-core\src\main\java\com\rapidminer\operator\**/*.java")

def findPrecondition(ListFile):
    count=0

    for File in ListFile:
        file1 = open(File, 'r')
        body = file1.read()
        startIndex=body.strip().find(".addPrecondition")
        while (startIndex >= 0):
            stringToEvaluate=body[startIndex:len(body)]
            endIndex=findEndIndex(stringToEvaluate)

            count=count+1
            body=body[startIndex+endIndex:]
            startIndex=body.strip().find(".addPrecondition")
        file1.close()
        


In [72]:
def preconditionDict(ListFile):
    count=0
    dictList = {}
    for File in ListFile:
        file1 = open(File, 'r')
        body = file1.read()
        startIndex=body.strip().find(".addPrecondition")
        
            
        while (startIndex >= 0):
            override= "Not Overrode"
            stringToEvaluate=body[startIndex:len(body)]
            startNamePrecondition = stringToEvaluate.strip().find("new ")
            if startNamePrecondition<50:
                startIndex+=startNamePrecondition+4
                stringToEvaluate=body[startIndex:len(body)]
                endIndex=findNextParenthesis(stringToEvaluate)
            else:
                endIndex = startIndex+51

            if(endIndex-startIndex<50):
                stringToEvaluate=body[startIndex-(startNamePrecondition+4):len(body)]
                endPreconditionIndex=findEndIndex(stringToEvaluate)
                if body[startIndex-(startNamePrecondition+4):startIndex+endPreconditionIndex].strip().find("@Override")>0:
                    override="Overrode"
                if File in dictList:
                    dictList[File].append([body[startIndex:startIndex+endIndex],override])
                else :
                    dictList[File]=[[body[startIndex:startIndex+endIndex],override]]
            count=count+1
            if(endIndex != None):
                body=body[startIndex+endIndex:]
            else:
                body=body[startIndex:]
            startIndex=body.strip().find(".addPrecondition")
        file1.close()


In [73]:
preconditionDict(ListFile)

In [74]:
def findOpInLine(string):
    endIndex=findEndIndexLine(string)
    print(string[:endIndex])

In [75]:
def findOpAndPort(string, string2):
    startIndex = string.find(string2)+len(string2)+2
    if (string.find(string2) >= 0):
        endIndex = findEndIndexLine(string[startIndex:])
        return string[startIndex:startIndex+endIndex]

In [76]:
# Récupération dans les processus des connexions

ListFile=glob2.glob(r".\rapidminer-studio-modular-master\rapidminer-studio-core\src\main\resources\com\rapidminer\resources\samples\processes\**/*.rmp")
#ListFile=glob2.glob(r".\projects_openML\**/*.xml")
dictGraph = {}

for File in ListFile:
    file1 = open(File, 'r')
    body = file1.read()
    startIndex=body.strip().find("from_op")
        
    #if (startIndex>=0):
        #print(File)
            
    while (startIndex >= 0):
        stringToEvaluate=body[startIndex:len(body)]
        endIndex=findEndIndexXML(stringToEvaluate)
        #print(body[startIndex:startIndex+endIndex-1]+"\n\n")
        
        from_op = findOpAndPort(body[startIndex:startIndex+endIndex-1], "from_op")
        from_port = findOpAndPort(body[startIndex:startIndex+endIndex-1], "from_port")
        to_op = findOpAndPort(body[startIndex:startIndex+endIndex-1], "to_op")
        to_port = findOpAndPort(body[startIndex:startIndex+endIndex-1], "to_port")
        
        if (to_op != None ):
            if (from_op.find(' (2)')>0):
                print(from_op)
                from_op=from_op[:from_op.find(" (2)")]
                
            if (to_op.find(" (2)")>0):
                print(to_op)
                to_op=to_op[:to_op.find(" (2)")]
                
            if (from_op.find(" (3)")>0):
                print(from_op)
                from_op=from_op[:from_op.find(" (3)")]
            
            if (to_op.find(" (3)")>0):
                print(to_op)
                to_op=to_op[:to_op.find(" (3)")]   
                 
            if (from_op.find("_1")>0):
                print(from_op)
                from_op=from_op[:from_op.find("_1")]
                
            if (to_op.find("_1")>0):
                print(to_op)
                to_op=to_op[:to_op.find("_1)")]
                
            if (from_op in dictGraph):
                dictGraph[from_op].append(to_op)
            else:
                dictGraph[from_op] = [to_op]
        body=body[startIndex+endIndex:]
        startIndex=body.strip().find("from_op")
    file1.close()



Retrieve (2)
ExampleSetGenerator (2)
ExampleSetGenerator (3)
ExampleFilter (2)
Select Attributes (2)
Select Attributes (2)
IOMultiplier_1
IOMultiplier_1
IOMultiplier_1
ExampleFilter (2)
Select Attributes (2)
Select Attributes (2)
IOMultiplier_1
IOMultiplier_1
IOMultiplier_1
Numerical2Polynominal (2)
Numerical2Polynominal (2)
ChangeAttributeName (2)
ChangeAttributeName (2)
IOStorer (2)
GuessValueTypes (2)
GuessValueTypes (2)
Retrieve (2)
ModelApplier (2)
RegressionPerformance (2)
IOMultiplier_1
IOMultiplier_1
Cross Validation (2)
IOMultiplier_1
Cross Validation (2)
Retrieve (2)
Training (2)


In [77]:
print(dictGraph)

{'Retrieve': ['DecisionTree', 'LinearRegression', 'Neural Net', 'NearestNeighbors', 'NaiveBayes', 'JMySVMLearner', 'MyKLRLearner', 'FrequencyDiscretization', 'Bagging', 'MissingValueReplenishment', 'DecisionTree', 'Apply Model', 'Preprocessing', 'NoiseGenerator', 'Cross Validation', 'Tree2RuleConverter', 'Cross Validation', 'Nominal2Binominal', 'Normalize', 'IdTagging', 'Sampling', 'FrequencyDiscretization', 'MinimalEntropyPartitioning', 'ExampleFilter', 'Preprocessing', 'NoiseGenerator', 'RemoveCorrelatedFeatures', 'Generation', 'NominalFeatureIterator', 'IdTagging', 'MissingValueReplenishment', 'JMySVMLearner', 'Cross Validation', 'MissingValueReplenishment', 'Cross Validation', 'SimpleValidation', 'Validation', 'NaiveBayes', 'ModelApplier', 'Cross Validation', 'RemoveCorrelatedFeatures', 'RemoveUselessAttributes', 'Normalization', 'PCA', 'GHA', 'FastICA', 'PCA', 'Relief', 'FS', 'PCAWeighting', 'EvolutionaryWeighting', 'InitialWeights', 'YAGGA', 'AttributeConstructionsLoader', 'IdTag

In [78]:
with open('dictGraph.json', 'w') as convert_file:
     convert_file.write(json.dumps(dictGraph))